In [ ]:

LONDON_URL="https://results.tcslondonmarathon.com/2024/?page={}&event=MAS&num_results=1000&pid=list&pidp=start&search%5Bsex%5D={}&search%5Bage_class%5D=%25"
CHICAGO_URL= "https://results.chicagomarathon.com/2023/?page={}&event=MAR&num_results=1000&pid=list&pidp=start&search%5Bsex%5D={}&search%5Bage_class%5D=%25"

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
df_final = None

for sex in ['M', 'W', 'D']:
    page_number=1

    while True:
        print(f"Retrieving page {page_number} for sex {sex}")
        res = requests.get(CHICAGO_URL.format(page_number, sex))
        page = BeautifulSoup(res.text)

        rows=page.find_all("li", class_='row')

        if len(rows)==2 and rows[1].find('div', class_='alert'):
            break

        rows2 = [r.find_all("div", class_='list-field') for r in rows]

        df = pd.DataFrame([[d.text for d in data] for data in rows2])
        df.insert(0,'Sex',sex)

        if df_final is None:
            df_final = df
        else:
            df_final = pd.concat([df_final, df])

        page_number+=1





In [ ]:
df_final.shape

In [ ]:
df_final.head()

In [ ]:

df_final[3] = df_final[3].str.replace('Division','')
df_final[2] = df_final[2].str.replace('BIB','') 

df_final[5] = df_final[5].str.replace('HALF','')
df_final[6] = df_final[6].str.replace('Finish','')

In [ ]:
df_final=df_final.reset_index(drop=True)
df_final=df_final[df_final[3]!='BIBBIB']

In [ ]:
df_final[['Sex',3,6]].to_csv('chicago_2023.csv', header=['sex','age','time'],index=False)

In [ ]:
df=df_final[['Sex',3,6]]
df.columns = ['sex','age','time']

In [ ]:
df_final[df_final[3]=='–']

In [ ]:
df.groupby('age').count()